In [1]:
# 导入同花顺包和登录同花顺
import datetime as dt
import pandas as pd
from numpy import *
from iFinDPy import *
THS_iFinDLogin("gdzq6888", "199212")

C:\Users\Xuwei\anaconda3\Lib\site-packages\iFinDPy.pth


0

In [2]:
from datetime import datetime
from calendar import monthrange
import numpy as np

In [3]:
# 定义给stock code分组的方程
def split(list_a, chunk_size):
    for i in range(0, len(list_a), chunk_size):
        yield list_a[i:i + chunk_size]

In [4]:
# 定义计算相关性平均值的方程
def find_upper_triangle(matrix):
    mask = np.triu(np.ones(matrix.shape), k=1)
    return matrix[mask.astype(bool)]

def solve(a):
    return find_upper_triangle(a)

# 多头alpha

## 个股截面离散度

In [28]:
# 输入日期格式为’年-月‘
def individual_stock_dispersion(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]
    # 获取中证800指数成分
    data = THS_DP('index_weight',month_end+';000906.SH','date:Y,thscode:Y,security_name:Y,weight:Y').data
    # 提取股票代码
    stock_code = list(data['thscode'])
    # 将股票代码分为4份
    stock_code2 = list(split(stock_code,200))
    #获得成分股每日价格
    data = pd.DataFrame()
    for i in stock_code2:
        a = THS_HQ(i,'close','',month_end_10years,month_end).data
        data=pd.concat([data,a])
    # 加入月，日
    data['year'] = data['time'].apply(lambda x: x.split('-')[0])
    data['month'] = data['time'].apply(lambda x: x.split('-')[1])
    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()

    # 计算日标准差
    std = data.groupby(['time'])['return'].std().reset_index()
    std = std.rename(columns={'return':'daily std'})
    std['month'] = std['time'].str[:7]

    # 得到离散度
    std_monthly = std.groupby('month')['daily std'].mean().reset_index()
    std_monthly = std_monthly.rename(columns={'daily std':'monthly std'})

    # 计算高/低离散度
    dispersion = list(std_monthly['monthly std'])[-1]/mean(list(std_monthly['monthly std']))

    return dispersion


In [30]:
individual_stock_dispersion('2023-01')

0.7584548441638147

## 行业截面离散度

In [10]:
def industry_dispersion(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]

    # 获取一级行业指数回报
    data=THS_HQ('CI005001.CI,CI005002.CI,CI005003.CI,CI005004.CI,CI005005.CI,CI005006.CI,CI005007.CI,CI005008.CI,CI005009.CI,CI005010.CI,CI005011.CI,CI005012.CI,CI005013.CI,CI005014.CI,CI005015.CI,CI005016.CI,CI005017.CI,CI005018.CI,CI005019.CI,CI005020.CI,CI005021.CI,CI005022.CI,CI005023.CI,CI005024.CI,CI005025.CI,CI005026.CI,CI005027.CI,CI005028.CI,CI005029.CI,CI005030.CI','close','',month_end_10years,month_end).data

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()

    # 计算日标准差
    std = data.groupby(['time'])['return'].std().reset_index()
    std = std.rename(columns={'return':'daily std'})
    std['month'] = std['time'].str[:7]

    # 得到离散度
    std_monthly = std.groupby('month')['daily std'].mean().reset_index()
    std_monthly = std_monthly.rename(columns={'daily std':'monthly std'})

    # 计算高/低离散度
    dispersion = list(std_monthly['monthly std'])[-1]/mean(list(std_monthly['monthly std']))

    return  dispersion


In [11]:
industry_dispersion('2023-01')

0.9762148769812952

## 个股相关性

In [12]:
# 输入日期格式为’年-月‘
def stock_correlation(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]
    # 获取中证800指数成分
    data = THS_DP('index_weight',month_end+';000906.SH','date:Y,thscode:Y,security_name:Y,weight:Y').data
    # 提取股票代码
    stock_code = list(data['thscode'])
    # 将股票代码分为4份
    stock_code2 = list(split(stock_code,200))
    #获得成分股每日价格
    data = pd.DataFrame()
    for i in stock_code2:
        a = THS_HQ(i,'close','',month_end_10years,month_end).data
        data=pd.concat([data,a])

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()
    data['month'] = data['time'].str[:7]

    # 计算相关性
    corr_list=list()
    for k in list(unique(data['month'])):
        df = pd.DataFrame()
        test = data[data['month']==k]
        for i in list(unique(test['thscode'])):
            a = test[test['thscode'] == i].reset_index()
            a = a[['return']]
            a = a.rename(columns={'return':i})
            df = pd.concat([df,a],axis=1)
        corr_matrix = df.corr().fillna(0).to_numpy()
        mean_correlation = mean(solve(corr_matrix))
        corr_list.append(mean_correlation)
    correlation_mean = mean(corr_list)

    correlation = corr_list[-1]/correlation_mean

    return correlation



In [13]:
stock_correlation('2023-01')

0.3746244164380103

## 行业相关性

In [11]:
def industry_correlation(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]

    # 获取一级行业指数回报
    data=THS_HQ('CI005001.CI,CI005002.CI,CI005003.CI,CI005004.CI,CI005005.CI,CI005006.CI,CI005007.CI,CI005008.CI,CI005009.CI,CI005010.CI,CI005011.CI,CI005012.CI,CI005013.CI,CI005014.CI,CI005015.CI,CI005016.CI,CI005017.CI,CI005018.CI,CI005019.CI,CI005020.CI,CI005021.CI,CI005022.CI,CI005023.CI,CI005024.CI,CI005025.CI,CI005026.CI,CI005027.CI,CI005028.CI,CI005029.CI,CI005030.CI','close','',month_end_10years,month_end).data

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()
    data['month'] = data['time'].str[:7]

    corr_list=list()
    for k in list(unique(data['month'])):
        df = pd.DataFrame()
        test = data[data['month']==k]
        for i in list(unique(test['thscode'])):
            a = test[test['thscode'] == i].reset_index()
            a = a[['return']]
            a = a.rename(columns={'return':i})
            df = pd.concat([df,a],axis=1)
        corr_matrix = df.corr().fillna(0).to_numpy()
        mean_correlation = mean(solve(corr_matrix))
        corr_list.append(mean_correlation)
    correlation_mean = mean(corr_list)

    correlation = corr_list[-1]/correlation_mean

    return correlation

In [12]:
industry_correlation('2023-01')

array([[ 1.        ,  0.49747065,  0.48511906,  0.60918228,  0.68026759,
         0.43045519,  0.59230266,  0.2289542 , -0.04522096,  0.21011445,
         0.31640072,  0.05988549, -0.1772572 ,  0.10185522, -0.02440117,
         0.08891448,  0.30731479,  0.04966744, -0.00860504,  0.39075855,
        -0.0421191 ,  0.19092591,  0.07838167,  0.35053425, -0.06522161,
         0.17957348, -0.01491843, -0.20542677,  0.15966008,  0.03473208],
       [ 0.49747065,  1.        ,  0.32760207,  0.2805951 ,  0.28356738,
        -0.07232732,  0.16969964, -0.13446294, -0.46837168, -0.35416704,
        -0.01682199, -0.27374724, -0.36199985, -0.4636714 , -0.14738244,
        -0.19488837, -0.22171285, -0.11112524, -0.26732262, -0.09218069,
        -0.05536647,  0.03271075, -0.11035194, -0.01284745, -0.23031597,
         0.08387587, -0.2432012 , -0.36102364, -0.08834108, -0.11686695],
       [ 0.48511906,  0.32760207,  1.        ,  0.50830141,  0.62366719,
         0.61820479,  0.45233423, -0.1713899 , -0

## 市场流动性

In [7]:
# 取得中证800的涨跌幅
data = THS_HQ('000906.SH','changeRatio,turnoverRatio','','2013-02-02','2023-02-02').data

In [9]:
data['日低流动性'] = data['changeRatio']/data['turnoverRatio']

In [10]:
data

,time,thscode,changeRatio,turnoverRatio,日低流动性
0,2013-02-04,000906.SH,-0.015417,0.930203,-0.016573
1,2013-02-05,000906.SH,0.900610,0.828819,1.086618
2,2013-02-06,000906.SH,0.266765,0.632775,0.421580
3,2013-02-07,000906.SH,-0.363568,0.650937,-0.558530
4,2013-02-08,000906.SH,0.654623,0.583527,1.121838
...,...,...,...,...,...
2422,2023-01-20,000906.SH,0.650587,0.585588,1.110997
2423,2023-01-30,000906.SH,0.483076,0.765846,0.630775
2424,2023-01-31,000906.SH,-0.768684,0.639636,-1.201752
2425,2023-02-01,000906.SH,1.011315,0.722341,1.400052


In [18]:
data1

,index,close
0,0.0,1711.2
1,0.6,1723.8
2,1.2,1720.2
3,1.8,1729.2
4,2.4,1744.2
...,...,...
208,124.8,1738.8
209,125.4,1724.4
210,126.0,1715.4
211,126.6,1735.2


In [19]:
data2

,index,close
0,0.00,911.50
1,0.25,923.25
2,0.50,921.75
3,0.75,903.50
4,1.00,902.50
...,...,...
166,41.50,988.00
167,41.75,971.50
168,42.00,960.25
169,42.25,974.75
